In [1]:
import numpy as np
import matplotlib.pyplot as plt
import epics as ep
import h5py as h5
from time import sleep

In /sf/photo/miniconda/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /sf/photo/miniconda/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /sf/photo/miniconda/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


## User inputs

In [8]:
cameraString = 'SAROP21-PPRM102:'
AdjustName = 'SARFE10-OAPU044:'
SaveFN = cameraString[:-1]+'switch.h5'
numshots =30

## grab image and plot

In [9]:
def getImg(cameraString,AdjustName,numshots):
    imagePV =cameraString+'FPICTURE'
    widthPV =cameraString+'WIDTH'
    heightPV =cameraString+'HEIGHT'
    screenPOSPV =cameraString+'MOTOR_PROBE.RBV'
    
    if AdjustName == 'SATFE10-OAPU040:':
        Adjust_XPV = AdjustName+'MOTOR_X.VAL'
        Adjust_YPV = AdjustName+'MOTOR_Y.VAL'
        Adjust_WPV = AdjustName+'MOTOR_W.VAL'
        Adjust_HPV = AdjustName+'MOTOR_H.VAL'
        
        Adjust_X = ep.caget(Adjust_XPV)
        Adjust_Y = ep.caget(Adjust_YPV)
        Adjust_W = ep.caget(Adjust_WPV)
        Adjust_H = ep.caget(Adjust_HPV)        

    width = int(ep.caget(widthPV))
    height = int(ep.caget(heightPV))
    screenPOS = ep.caget(screenPOSPV)
    tmp_Img = []
    for i in range(0,numshots):
        tmp_Img.append(ep.caget(imagePV)[0:height*width].reshape([height, width]))
        sleep(0.3)
    tmp_Img = np.asarray(tmp_Img)
    Img = {
        "images": tmp_Img,
        "mean": tmp_Img.mean(axis=0),
        "Camera name": cameraString,
        "Screen position": screenPOS,
        "Width": width,
        "Height": height,
#         "AdjustName":AdjustName,
#         "Adjust_X":Adjust_X,
#         "Adjust_Y":Adjust_X,
#         "Adjust_W":Adjust_W,
#         "Adjust_H":Adjust_H
    }
    return Img

In [10]:
Img = getImg(cameraString,AdjustName,numshots)

In [11]:
with h5.File(SaveFN,'w-') as fh:
    for k,v in Img.items():
        fh[k] = v